In [30]:
import pandas as pd
import mysql.connector
import warnings

def fetch_data(query):
    # Suppress the UserWarning
    warnings.simplefilter(action='ignore', category=UserWarning)
    
    # Establish the connection
    cnx = mysql.connector.connect(user='root', host='127.0.0.1', database='mimnseoksong')
    
    # Fetch the data into a DataFrame
    df = pd.read_sql(query, cnx)
    
    # Close the connection
    cnx.close()
    
    # Return the DataFrame
    return df


How does the data look like?

In [31]:
fetch_data("""
SELECT * 
FROM USER_DETAILS
""")

,user_id,username,first_name,last_name,gender,password,status
0,1,rogers63,david,john,Female,e6a33eee180b07e563d74fee8c2c66b8,1
1,2,mike28,rogers,paul,Male,2e7dc6b8a1598f4f75c3eaa47958ee2f,1
2,3,rivera92,david,john,Male,1c3a8e03f448d211904161a6f5849b68,1
3,4,ross95,maria,sanders,Male,62f0a68a4179c5cdd997189760cbcf18,1
4,5,paul85,morris,miller,Female,61bd060b07bddfecccea56a82b850ecf,1
...,...,...,...,...,...,...,...
95,96,smith9,miller,morris,Male,97ee0765b9c05d35b53769a3c4133b13,1
96,97,miller73,chrishaydon,morgan,Male,6c4283471ace6b4af590c180bd13b1bf,1
97,98,michael44,cooper,maria,Female,dd4d053a12a3d8450166dba9177bac2c,1
98,99,michael36,miller,cooper,Female,36ab21ccb2a64acd5351bbb59753df9d,1


How many males and females are there?

In [32]:
fetch_data("""
SELECT SUM(CASE WHEN gender = 'Male' THEN 1 ELSE 0 END) as MaleCount,
SUM(CASE WHEN gender = 'Female' THEN 1 ELSE 0 END) as FemaleCount
FROM USER_DETAILS
""")




,MaleCount,FemaleCount
0,51.0,49.0


Which two last names appear most often, and how many times do they each occur?

In [33]:
fetch_data("""
SELECT last_name, COUNT(last_name) as frequency
FROM USER_DETAILS
GROUP BY last_name
ORDER BY frequency DESC
LIMIT 2
""")

,last_name,frequency
0,brown,8
1,sanders,7


Is there any name that is associated with both male and female genders?

In [34]:
fetch_data("""
SELECT distinct a.last_name
FROM USER_DETAILS as a
Join USER_DETAILS as b ON a.last_name = b.last_name
WHERE a.gender <> b.gender
""")

,last_name
0,john
1,paul
2,sanders
3,miller
4,michael
5,mike
6,maria
7,bell
8,brown
9,james


We can also identify their first names as well.

In [35]:
fetch_data("""
SELECT DISTINCT a.first_name, a.last_name, a.gender
FROM USER_DETAILS as a
Join USER_DETAILS as b ON a.last_name = b.last_name
WHERE a.gender <> b.gender
""")

,first_name,last_name,gender
0,rogers,john,Male
1,david,john,Male
2,bell,paul,Female
3,sanders,paul,Female
4,bell,john,Female
...,...,...,...
79,morris,chrishaydon,Female
80,rogers,chrishaydon,Female
81,sanders,mike,Male
82,brooks,mike,Male


How long is each password?

In [36]:
fetch_data("""
SELECT user_id, LENGTH(password) as 'length of password'
FROM USER_DETAILS
""")

,user_id,length of password
0,1,32
1,2,32
2,3,32
3,4,32
4,5,32
...,...,...
95,96,32
96,97,32
97,98,32
98,99,32


Do they all have 32 characters?

In [37]:
fetch_data("""
SELECT (CASE WHEN count(user_id)=100 THEN 'YES' ELSE 'NO' END) as 'Do they all have 32 characters?'
FROM USER_DETAILS
WHERE LENGTH(password)=32
""")

,Do they all have 32 characters?
0,YES


Are there any users who set their first or last name in their user id?

In [38]:
fetch_data("""
SELECT 
    CASE 
        WHEN EXISTS (
            SELECT 1
            FROM USER_DETAILS
            WHERE user_id LIKE CONCAT('%', first_name, '%')
            OR user_id LIKE CONCAT('%', last_name, '%')
        ) THEN 'YES'
        ELSE 'NO SUCH PERSON'
    END AS result;
""")

,result
0,NO SUCH PERSON
